# Working with Vector data

https://github.com/yeesian/ArchGDAL.jl/blob/master/docs/src/features.md <br>
https://github.com/JuliaGeo/Shapefile.jl

I have focused on raster data in this git hub repo, but ArchGDAL does have some excellent support for vector operations. This notebook is only partially completed. While it does write out a point shapefile it does not write the projection (.prj) file. I hope it is of use. Please do refer to the links supplied in this notebook for further information. Please be aware at ArchGDAL may have upgraded a version.

In [1]:
using ArchGDAL
filepath = "Boundary_isle.shp"
ArchGDAL.registerdrivers() do
    ArchGDAL.read(filepath) do dataset
        println(dataset)
        println(typeof(dataset))
    end
end

GDAL Dataset (Driver: ESRI Shapefile/ESRI Shapefile)
File(s): 
  Boundary_isle.shp
  Boundary_isle.shx
  Boundary_isle.dbf
  Boundary_isle.cpg
  Boundary_isle.prj

Number of feature layers: 1
  Layer 0: Boundary_isle (wkbPolygon)

ArchGDAL.Dataset


Investigate the data in the shapefile

In [2]:
using ArchGDAL
filepath = "Boundary_isle.shp"
ArchGDAL.registerdrivers() do
    ArchGDAL.read(filepath) do dataset
        layer = ArchGDAL.getlayer(dataset, 0)
        featuredefn = ArchGDAL.getlayerdefn(layer)
        println("how many fields?")
        println(ArchGDAL.nfield(featuredefn))
    end
end

how many fields?
1


we only have 1 field in this shapefile (note that field count is 1 but first field is 0) <br>
ref :https://github.com/yeesian/ArchGDAL.jl/blob/master/docs/src/features.md

In [3]:
using ArchGDAL
filepath = "Boundary_isle.shp"
ArchGDAL.registerdrivers() do
    ArchGDAL.read(filepath) do dataset
        layer = ArchGDAL.getlayer(dataset, 0)
        featuredefn = ArchGDAL.getlayerdefn(layer)
        println("how many fields?")
        println(ArchGDAL.nfield(featuredefn))
        fielddefn = ArchGDAL.getfielddefn(featuredefn, 0)
        println("field name is?")
        println(ArchGDAL.getname(fielddefn))
            
    end
end

how many fields?
1
field name is?
ID


If we did have multiple fields this is how to do it

In [4]:
using ArchGDAL
filepath = "Boundary_isle.shp"
ArchGDAL.registerdrivers() do
    ArchGDAL.read(filepath) do dataset
        layer = ArchGDAL.getlayer(dataset, 0)
        featuredefn = ArchGDAL.getlayerdefn(layer)
        println("how many fields?")
        println(ArchGDAL.nfield(featuredefn))
        for i = 0:((ArchGDAL.nfield(featuredefn))-1)
            fielddefn = ArchGDAL.getfielddefn(featuredefn, i)
            println("field name is?")
            println(ArchGDAL.getname(fielddefn))
        end
            
    end
end

how many fields?
1
field name is?
ID


We only have 1 field and we only have 1 feature. If we wanted to get the geometry of that feature we would use this code

In [5]:
using ArchGDAL
filepath = "Boundary_isle.shp"
ArchGDAL.registerdrivers() do
    ArchGDAL.read(filepath) do dataset
        layer = ArchGDAL.getlayer(dataset, 0)
        ArchGDAL.getfeature(layer, 0) do feature
        println(ArchGDAL.getgeomfield(feature, 0))
        end
    end
end

Geometry: POLYGON ((600140 5603280,600140 5626000,638610 562 ... 280))


Now that we have the geometry we can peforming spatial processing. In the example below lets get the centroid. <br>
ref: https://github.com/yeesian/ArchGDAL.jl/blob/master/docs/src/geometries.md

In [6]:
using ArchGDAL
filepath = "Boundary_isle.shp"
ArchGDAL.registerdrivers() do
    ArchGDAL.read(filepath) do dataset
        layer = ArchGDAL.getlayer(dataset, 0)
        ArchGDAL.getfeature(layer, 0) do feature
        geometry_pol = (ArchGDAL.getgeomfield(feature, 0))
        println(geometry_pol)
        ArchGDAL.centroid(geometry_pol)
        end
    end
end

Geometry: POLYGON ((600140 5603280,600140 5626000,638610 562 ... 280))


Geometry: POINT (619375 5614640)

Finally lets write this centroid out to a shapefile. I didn't find a way to write out a prj file with the associated .shp, .dbf .shx files. At the moment you will need to copy the prj from the input.

In [7]:
using ArchGDAL; const AG = ArchGDAL
using GDAL
filepath = "Boundary_isle.shp"
    
AG.registerdrivers() do
    
    AG.read(filepath) do dataset
        layer2 = AG.getlayer(dataset, 0)
        spatialref = AG.getspatialref(layer2)
        println(spatialref)
        
        AG.getfeature(layer2, 0) do feature
        geometry_pol = (AG.getgeomfield(feature, 0))
        println(geometry_pol)
        geom = AG.centroid(geometry_pol) ## get the centroid
        println(geom)


            AG.create("pointshapeOut.shp", "ESRI Shapefile") do datasetOut
                
                layer = AG.createlayer(datasetOut, "point_out", geom=GDAL.wkbPoint)
                
                featuredefn = AG.getlayerdefn(layer)
                geomfielddefn = ArchGDAL.getgeomfielddefn(featuredefn, 0)
                AG.setspatialref!(geomfielddefn, spatialref)  ## doesn't seem to work?
                
            
                AG.createfielddefn("Name", GDAL.OFTString) do fielddefn
                    AG.setwidth!(fielddefn, 32)
                    AG.createfield!(layer, fielddefn, true)
                end
                
                featuredefn = AG.getlayerdefn(layer)
                               
                AG.createfeature(featuredefn) do feature
                    
                    
                    AG.setfield!(feature, AG.getfieldindex(feature, "Name"), "centre point")
                                        
                    AG.setgeomdirectly!(feature, AG.unsafe_createpoint(619375, 5614640)) ## hardcoded - need to change
                    AG.createfeature!(layer, feature)
                
               
                AG.destroy(feature)
                end
             
end
        end
end
end


Spatial Reference System: +proj=utm +zone=30 +datum=WGS84 +un ... defs 
Geometry: POLYGON ((600140 5603280,600140 5626000,638610 562 ... 280))
Geometry: POINT (619375 5614640)


Ptr{Nothing} @0x0000000000000000